In [ ]:
import csv
from typing import Callable, Dict, Tuple

import matplotlib.pyplot as plt
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler

In [ ]:
with open('../../data/aggregate.csv', 'r') as f:
    all_rows = list(csv.DictReader(f))

In [ ]:
def row_duration(row: Dict[str, str]) -> float:
    """Get the duration of a row (in hours)."""
    start_hr, start_min = row['start_time'].split(':')
    end_hr, end_min = row['end_time'].split(':')
    elapsed = float(end_hr) - float(start_hr)
    if elapsed < 0:
        # Assume at most one day difference.
        elapsed += 24
    elapsed += (float(end_min) - float(start_min)) / 60
    return elapsed

def row_usage(row: Dict[str, str]) -> float:
    return float(row['usage']) / row_duration(row)

In [ ]:
def model_mse(model: Callable[[Dict[str, str]], float]) -> float:
    """Get the hourly usage prediction MSE for a prediction model."""
    errors = [(model(x) - row_usage(x)) ** 2 for x in all_rows]
    return float(np.mean(errors))

mean_usage = np.mean([float(x['usage']) / row_duration(x) for x in all_rows])
print('MSE of mean prediction:', model_mse(lambda _: mean_usage))

In [ ]:
hourly_usages = {i: [row for row in all_rows if int(row['start_time'].split(':')[0]) == i]
                 for i in range(24)}
hourly_avgs = {i: np.mean([row_usage(x) for x in rows]) for i, rows in hourly_usages.items()}
hourly_model = lambda row: hourly_avgs[int(row['start_time'].split(':')[0])]

hourly_mse = model_mse(hourly_model)
print('MSE of hourly prediction:', hourly_mse)

plt.figure('Average Hourly Usage')
plt.bar(hourly_avgs.keys(), hourly_avgs.values())
plt.xlabel('Hour')
plt.ylabel('Mean usage (kW)')
plt.show()

In [ ]:
def fit_linear_model(*fields: str, residual: bool = False) -> Tuple[
    Callable[[Dict[str, str]], float], Dict[str, float]
]:
    def row_values(x: Dict[str, str]) -> np.ndarray:
        return np.array([float(x[k]) for k in fields])

    data_inputs = np.stack([row_values(x) for x in all_rows], axis=0)
    data_outputs = np.array([row_usage(x) - (hourly_model(x) if residual else 0) for x in all_rows])

    scaler = StandardScaler()
    scaler.fit(data_inputs)

    model = LinearRegression()
    model.fit(data_inputs, data_outputs)

    def model_fn(row: Dict[str, str]) -> float:
        res = float(model.predict(row_values(row)[None])[0])
        if residual:
            res += hourly_model(row)
        return res

    return model_fn, dict(zip(fields, model.coef_.tolist()))

In [ ]:
weather_fields = [
    'temp',
    'cloudcover',
    'wind_speed',
    'precip_inches',
    'humidity',
    'pressure',
    'uv_index',
]
linear_mses = [model_mse(fit_linear_model(field)[0]) for field in weather_fields]
residual_mses = [model_mse(fit_linear_model(field, residual=True)[0]) for field in weather_fields]

ys, xs = zip(*sorted(zip(linear_mses, weather_fields)))
ys_residual = [residual_mses[weather_fields.index(x)] for x in xs]
ax = plt.figure()
plt.bar(xs, ys, label='Base model')
plt.bar(xs, ys_residual, label='Hourly residual')
plt.xticks(rotation='vertical')
plt.ylim(min(ys_residual)*0.9, model_mse(lambda _: mean_usage))
plt.xlabel('Variable')
plt.ylabel('Linear regression MSE')
plt.legend()
plt.show()

In [ ]:
residual_temp_model, coeffs = fit_linear_model('temp', residual=True)
print('residual temp coeffs', coeffs)
print('residual temp MSE:', model_mse(residual_temp_model))

residual_weather_model, coeffs = fit_linear_model(*weather_fields, residual=True)
print('residual weather coeffs', coeffs)
print('residual weather MSE:', model_mse(residual_weather_model))